Lecture 11: Smoothing a noisy image using diffusion equation, the key here is to know
    when the smoothing should be stopped so as not to remove too much of image content. It can be shown that linear filtering with Gaussian is equivalent to linear diffusion.

In [ ]:
%pylab

In [ ]:
from skimage.color import rgb2gray
from scipy.integrate import solve_ivp

A = imread("./Sharbat_Gula.jpg")

Abw = rgb2gray(A)
nx,ny = Abw.shape

noise = 0.5
An = Abw + noise*randn(nx,ny)

from scipy.sparse import spdiags,kron
x = linspace(0,1,nx)
dx = x[1]-x[0]
y = linspace(0,1,ny)
dy = y[1]-y[0]

e1x = ones(nx)
e1y = ones(ny)
Dx = spdiags([e1x,-2*e1x,e1x],[-1,0,1],nx,nx) / dx**2  # laplacian matrix in x dimension 
Dy = spdiags([e1y,-2*e1y,e1y],[-1,0,1],ny,ny) / dy**2

Ix = eye(nx)
Iy = eye(ny)
L = kron(Iy,Dx) + kron(Dy,Ix)     # constructs laplacian in two dimension using kron command

D = 0.0005   # diffusion constant
u0 = reshape(An,(nx*ny),'F')    # 'F':fortran-like(as matlab), 'C': C-like, should change L

def image_rhs(t,u,D,L):
    return D*L.dot(u)

t0 = 0
tf = 0.1
sol = solve_ivp(image_rhs,[t0,tf],u0,t_eval=arange(0,tf,tf/10),\
                args=(D,L))   # add more options

for i in range(len(sol.t)):
    imshow(reshape(sol.y[:,i],(nx,ny),'F'))
    pause(0.2)